# Descrizione del problema
Il problema riguarda lo **scheduling**. Sono dati $G$ ingegneri che lavorono per $L$ giorni. La realtà è composta da 3 elementi:
* **Features**: sono le funzionalità implementate nei prodotti;
* **Services**: sono i programmi che sono in esecuzione sui server;
* **Binaries**: modellano un server. Ogni *binary* può avere in esecuzione più servizi;

## Relazioni tra le entità
Una **Feature** è contraddistinta dal numero di utenti che riesce a soddisfare e da un insieme di servizi in cui deve essere implementata. Una **Feature** è disponibile ai clienti quando viene implementata in ogni servizio da cui dipende.
Un **Service** è presente su **uno e un solo binary**, ma su ogni **binary** è possibile inserire più **Services**.

### Lettura input da file
Come prima cosa vengono letti i dati input e si effettua il parsing della prima riga per avere la dimensione del problema.

In [8]:
import pathlib
from dataclasses import dataclass
from typing import List, Tuple, Dict, Set
from queue import SimpleQueue

fname = "data/f.in"

# lettura dei dati dal file di input
with open(fname, "r") as f:
    lines = f.readlines()
    
# parsing della prima riga: dimensione del problema
L, G, S, B, F, N = [int(k) for k in lines[0].strip().split(" ")]

# modellazione del sistema

score: int = 0
daily_schedule: SimpleQueue[Tuple[int, int]] = SimpleQueue()
@dataclass
class Binary:
    id: int
    services: List[int]
    working_on: int

binaries: List[Binary] =  [None] * B
service_to_binaries: Dict[str, int] = {}

@dataclass
class Service:
    id: int
    name: str
    binary: int
    impl_features: List[str]

services: List[Service] = []

@dataclass
class Feature:
    idx: int
    name: str
    difficulty: int
    users: int
    dep_services: List[List]
    binaries: Set[int]
    day_completed: int
    def populate_binaries(self):
        for (service, done) in self.dep_services:
            b = service_to_binaries[service]
            self.binaries.add(b)

    def is_finished(self) -> bool:
        for (service, completed) in self.dep_services:
            if not completed:
                return False
        return True

features: List[Feature] = []

@dataclass
class Engineer:
    id: int
    avail: int
    ops: List[Tuple[str, List[int]]]

    def add_op(self, op_name: str, day: int, duration: int, values: List[int]):
        assert(self.avail <= day)
        if self.avail < day:
            self.ops.append(("wait", [day - self.avail]))

        self.ops.append((op_name, values))
        self.avail = day + duration
        
engineers: List[Engineer] = []

for i in range(G):
    engineers.append(Engineer(i, 0, []))

idx = 0
for (service, binary) in [line.strip().split() for line in lines[1:S+1]]:
    bid = int(binary)
    b = binaries[bid]
    if b is None:
        b = Binary(bid, [idx], 0)
        binaries[bid] = b
    else:
        b.services.append(idx)

    s = Service(idx, service, bid, [])
    idx+=1
    services.append(s)
    service_to_binaries[service] = bid

idx = 0
for i, feature in enumerate(lines[1+S:]):
    # leggo solo le righe pari
    if i % 2 ==0:
        feature, num_services, difficulty, users = feature.strip().split(" ")
        s = [[s, False] for s in lines[1+S+1+i].split()]
        f = Feature(idx, feature, int(difficulty), int(users), s, set(), 0)
        f.populate_binaries()
        features.append(f)
        idx += 1

def process_timeline(day: int):
    while not daily_schedule.empty():
        elem = daily_schedule.get_nowait()
        if elem[0] < day:
            bid: int = elem[1]
            binaries[bid].working_on -=1
            assert(binaries[bid].working_on >= 0)

def implement_feature(e: int, day: int, f: int, b: int) -> int:
    assert(b >= 0 and b < B)
    assert(day >=0 and day < L)
    assert(f >= 0 and f < F)
    assert(e >= 0 and e < G)
    global score
    process_timeline(day)
    feature = features[f]
    was_finished = feature.is_finished()
    duration  = feature.difficulty + len(binaries[b].services) + binaries[b].working_on
    for i, (service, done) in  enumerate(feature.dep_services):
        if service_to_binaries[service] == b and not done:
            feature.dep_services[i][1] = True
            feature.day_completed = max(day + duration, feature.day_completed)
    
    if feature.is_finished() and not was_finished:
        print("done feature", f)
        score += feature.users * max(0, L-feature.day_completed) 

    engineers[e].add_op("impl", day, duration, [f, b])
    daily_schedule.put((day + duration, b))
    binaries[b].working_on+=1
    
    return duration

def move_service(e: int, day: int, s: int, dst: int) -> int:
    assert(s >= 0 and s < S)
    assert(day >= 0 and  day < L)
    assert(dst >= 0 and dst < B)
    assert(e >= 0 and e < G)

    process_timeline(day)
    service = services[s]
    old_b = service.binary
    duration  = max(len(binaries[old_b].services), len(binaries[dst].services))
    binaries[old_b].services.remove(service.id)
    binaries[dst].services.append(service.id)
    service.binary = dst
    engineers[e].add_op("move", day, duration, [s, dst])
    return duration

def new_binary(e: Engineer, day: int)-> int:
    assert(day >=0 and day < L)
    assert(e >= 0 and e < G)
    binaries.append(Binary(len(binaries), [], 0, False))
    e.add_op("new", day, N, [])
    return N

def wait(e: Engineer, day: int, duration: int) -> int:
    assert(day >=0 and day < L)
    e.add_op("wait", day, duration, [])
    return duration

features.sort(key=lambda x: x.users/(len(x.binaries)* x.difficulty), reverse=True)

f = 0
b = -1
for day in range(L):
    for engineer in engineers:
        if engineer.avail <= day:
            b+=1
            if b == B:
                b = 0
                f+=1
            if f == F:
                f = 0
            if features[f].is_finished():
                continue

            todo = False
            for (service, done) in features[f].dep_services:
                if not done and service_to_binaries[service] == b:
                    todo = True

            if todo:
                implement_feature(engineer.id, day, f, b)

working_engineers: List[Engineer] = [engineer for engineer in engineers if len(engineer.ops) > 0]

ofname = pathlib.Path(fname).with_suffix(".out")
with open(ofname, "w") as file:
    file.write(str(len(working_engineers)) + "\n")
    for engineer in working_engineers:
        file.write(str(engineer.id)+ "\n")
        for op in engineer.ops:
            line = op[0]+ " "
            if op[0] == "impl":
                line += features[op[1][0]].name + " " + str(op[1][1]) + "\n"
            elif op[0] == "wait":
                line += str(op[1][0]) + "\n"
            elif op[0] == "move":
                line += " ".join([str(k) for k in op[1]]) + "\n"
            else:
                raise Exception(f"unsupported operation {op[0]}")
                
            file.write(line)

# print(engineers, binaries, service_to_binaries, features, services, sep="\n")
print("score = ", score)
completed_features = [f for f in features if f.is_finished() and f.day_completed < L]
print("completed", len(completed_features), "features")
# record 158.421.404

done feature 0
done feature 1
done feature 2
done feature 3
done feature 4
done feature 5
done feature 6
done feature 7
done feature 8
done feature 9
done feature 10
done feature 11
done feature 12
done feature 13
done feature 14
done feature 15
done feature 16
done feature 17
done feature 18
done feature 19
done feature 20
done feature 21
done feature 22
done feature 23
done feature 24
done feature 25
done feature 26
done feature 27
done feature 28
done feature 29
done feature 30
done feature 31
done feature 32
done feature 33
done feature 34
done feature 35
done feature 36
done feature 37
done feature 38
done feature 39
done feature 40
done feature 41
done feature 42
done feature 43
done feature 44
done feature 45
done feature 46
done feature 47
done feature 48
done feature 49
done feature 50
done feature 51
done feature 52
done feature 53
done feature 54
done feature 55
done feature 56
done feature 57
done feature 58
done feature 59
done feature 60
done feature 61
done feature 62
do